In [1]:
from shapely.geometry import Polygon, Point, MultiPolygon, LineString
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Notebook overview

In this notebook we filter the processed extended clusters based on their shape and size. We use the same reqruiements as we did previously. 

**Input**: Cluster dictionary of extended polygons as created in notebook 2.

**Output**: Shapely file with extended polygons containing only rectangular clusters.

**Previous notebook**: 2. Point cloud processing, matching and cluster growing.

**Next notebook**: 5. Merge extended polygons.

# Loading data

In [2]:
# Load cluster dictionary
path = "../data/output/extension cluster dict.pkl"

with open(path, 'rb') as file:
    clusters = pickle.load(file)

# Plotting function

In [3]:
# Function to plot PC polygon (can be used if desired, not necessary)
def plot_PC_2D(PC_pol_dict, coords, intensity):
    x = PC_pol_dict[coords][:, 0]
    y = PC_pol_dict[coords][:, 1]
    plt.figure()
    plt.scatter(x, y, c=PC_pol_dict[intensity], cmap='viridis', vmin=0)
    plt.colorbar(label='Reflective index')  # Add colorbar to show gradient values
    plt.xlabel('X')
    plt.ylabel('Y')

    plt.grid(True)
    plt.axis('equal')
    plt.show()

# Filtering polygons

In [4]:
# Function to create shapely polygons out of the PC polygons
def to_shapely_polygon(clusters):
    for cluster_group in clusters:
        for cluster in cluster_group:
            pols = []
            for coor in cluster['clean_coords']:
                temp_point = Point(coor[0], coor[1])
                buff_point = temp_point.buffer(0.08)
                pols.append(buff_point)
            multi_pol = MultiPolygon(pols)
            cluster['Multi Polygon'] = multi_pol
            cluster['Polygon'] = multi_pol.convex_hull
    
    return clusters

In [5]:
clusters = to_shapely_polygon(clusters)

In [6]:
# Function to plot the polygons
def plot_polygon(polygon):
    # Extract polygon coordinates
    x_coords, y_coords = polygon.exterior.xy
    
    # Calculate aspect ratio based on the range of x and y coordinates
    x_range = max(x_coords) - min(x_coords)
    y_range = max(y_coords) - min(y_coords)
    aspect_ratio = y_range / x_range
    
    # Create the plot with the calculated aspect ratio
    fig, ax = plt.subplots(figsize=(5, 5 * aspect_ratio))  # Adjust the figsize as needed
    ax.set_aspect('equal')  # Ensure equal aspect ratio
    
    # Plot the polygon
    ax.plot(x_coords, y_coords, color='blue')
    
    # Show the plot
    plt.show()

In [7]:
# Function to check if a polygon is a crosswalk stripe
def is_rectangle(polygon):

    # We first check if the minimum rotated rectangle overlaps for at least 90% with the original polygon
    # This filters out shapes that are not square/rectangular
    min_rect = polygon.minimum_rotated_rectangle
    
    area = polygon.area/min_rect.area

    if area > .90:
        area_bool = True
        
    else:
        area_bool = False

    # We then check the lengths of the vertices of the polygon to see if they resemble a crosswalk stripe
    if area_bool == True:
        coords = min_rect.boundary.coords
        
        linestrings = [LineString(coords[k:k+2]) for k in range(len(coords) - 1)]

        lengths = []

        for line in linestrings:
            lengths.append(line.length)
        
        # Define the conditions
        condition1 = sum(1.4 <= length <= 7 for length in lengths)
        condition2 = sum(0.6 <= length <= 0.8 for length in lengths)

        # Check if the conditions are met
        if not (condition1 == 2 and condition2 == 2):
            area_bool = False

    
    return area, area_bool


In [8]:
# Function to merge rectangular clusters of one crosswalk polygon
def merge_clusters(clusters):

    polygons = []
    clean_coords = []
    clean_intensity = []
    coords = []
    intensity = []

    for cluster in clusters:
        polygons.append(cluster['Polygon'])
        coords.append(cluster['coordinates'])
        intensity.append(cluster['intensity'])
        clean_coords.append(cluster['clean_coords'])
        clean_intensity.append(cluster['clean_intensity'])
    
    PC_coords = np.concatenate(coords, axis=0)
    PC_intensity = np.concatenate(intensity, axis=0)
    PC_coords_clean = np.concatenate(clean_coords, axis=0)
    PC_intensity_clean = np.concatenate(clean_intensity, axis=0)
    
    multi_pol = MultiPolygon(polygons)
    
    pol = multi_pol.convex_hull

    pol_dict = {'CW_index': clusters[0]['CW_index'], 'coordinates': PC_coords, 'intensity': PC_intensity, 'clean_coords': PC_coords_clean, 'clean_intensity': PC_intensity_clean, 'polygon': pol}

    return pol_dict

In [9]:
# Function to filter out duplicates in the rectangle cluster dictionary
def filter_duplicates(data):
    seen_lengths = set()
    unique_items = []
    
    for item in data:
        length = len(item['clean_coords'])
        if length not in seen_lengths:
            seen_lengths.add(length)
            unique_items.append(item)
    
    return unique_items

In [10]:
# Function to check if the polygons are rectangles 
def check_rectangles(clusters):
    # Initialize list of rectangular clusters
    rect_clusters = []

    # Initialize list of final polygons
    polygon_dict = []

    for i in range(0, len(clusters)):

        merged = merge_clusters(clusters[i])

        # Initialize list to keep track of rectangular cluster dictionaries in a crosswalk
        cluster_list = []

        for cluster in clusters[i]:
            
            area, rect_bool = is_rectangle(cluster['Polygon'])
            
            if rect_bool is True:

                # Append cluster to list of rectangular clusters of the crosswalk
                cluster_list.append(cluster)        

        if len(cluster_list) > 0:

            rect_clusters.append(filter_duplicates(cluster_list))

            # Merge clusters
            pol = merge_clusters(cluster_list)

            # Add final polygon to polygon dict
            polygon_dict.append(pol)

    return polygon_dict, rect_clusters

In [11]:
polygon_dict, rect_clusters = check_rectangles(clusters)

In [12]:
# Save final polygons as gdf
path = "../data/output/filtered extended polygons.shp"
polygons_df = pd.DataFrame.from_records(polygon_dict)
polygons_df = polygons_df.drop(columns=['CW_index', 'coordinates', 'intensity', 'clean_coords', 'clean_intensity'])
polygons_gdf = gpd.GeoDataFrame(polygons_df, crs='epsg:28992', geometry='polygon')
polygons_gdf.to_file(path)